In [3]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM,Bidirectional,Dense,Input,Conv2D,MaxPool2D,Lambda,BatchNormalization,Reshape,Lambda
from keras.models import Model
import keras.backend as K
import cv2
import fnmatch

from keras.callbacks import ModelCheckpoint
from keras.activations import relu,sigmoid,softmax
from keras.utils import to_categorical
import os
import numpy as np

Using TensorFlow backend.
C:\Users\joyje\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\joyje\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\joyje\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\joyje\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [4]:
from keras.models import Sequential
from keras.layers import TimeDistributed
from keras.optimizers import Adam

In [5]:

import tensorflow as tf
import string

In [7]:
char_list=[]
char_list=string.ascii_letters+string.digits #list of all alphabets and digits
def encode_labels(txt):
    dig_list=[]
    for index, char in enumerate(txt):
        try:
            dig_list.append(char_list.index(char))
        except:
            print(char)
    return dig_list

In [8]:
file="C:/Users/joyje/Desktop/mnt/ramdisk/max/90kDICT32px"

train_img=[]
train_text=[]
train_input_length=[]
train_label_length=[]
orig_text=[]
valid_img=[]
valid_text=[]
valid_input_length=[]
valid_label_length=[]
valid_orig_text=[]
max_len=0
flag=0
i=1
for root,dirname,filenames in os.walk(file):
    for fname in fnmatch.filter(filenames,"*.jpg"):
        imsge=cv2.cvtColor(cv2.imread(os.path.join(root,fname)),cv2.COLOR_BGR2GRAY)
        w,h=imsge.shape
        #imsge=imsge.Reshape((32,128,1))
        
        if h>128 or w>32:
            continue
        if w<32:
            pad_zeros=np.ones((32-w,h))*255
            imsge=np.concatenate((imsge,pad_zeros))
        if h<128:
            pad_zeros=np.ones((32,128-h))*255
            imsge=np.concatenate((imsge,pad_zeros),axis=1)
        
        imsge=np.expand_dims(imsge,axis=2)
        imsge=imsge/255
        text=fname.split('_')[1]
        if len(text)>max_len:
            max_len=len(text)
        if i%5==0:
            valid_orig_text.append(text)
            valid_label_length.append(len(text))
            valid_input_length.append(63)
            valid_img.append(imsge)
            valid_text.append(encode_labels(text))
        else:
            orig_text.append(text)
            train_label_length.append(len(text))
            train_input_length.append(31)
            train_img.append(imsge)
            train_text.append(encode_labels(text))
        if i==200000:
            flag=1
            break
        i+=1
    if flag==1:
        break
train_pad=pad_sequences(train_text,maxlen=max_len,padding='post',value=len(char_list))
vaild_pad=pad_sequences(valid_text,maxlen=max_len,padding='post',value=len(char_list))

        





cnn=Sequential()
cnn.add(Conv2D(32,(3,3),padding='same',activation='relu'))
cnn.add(MaxPool2D((3,3)))
cnn.add(Conv2D(48,(3,3),padding='same',activation='relu'))
cnn.add(MaxPool2D((3,3)))
cnn.add(Conv2D(64,(5,5),padding='same',activation='relu'))
cnn.add(Conv2D(128,(5,5),padding='same',activation='relu'))
cnn.add(MaxPool2D((2,2)))
cnn.add(Conv2D(256,(5,5),padding='same',activation='relu'))
cnn.add(BatchNormalization())
cnn.add(Conv2D(512,(5,5),padding='same',activation='relu'))
cnn.add(BatchNormalization())
cnn.add(MaxPool2D(2,2))
cnn.add(Conv2D(512,(5,5),padding='same',activation='relu'))
model1=Sequential()
model1.add(TimeDistributed(cnn))
model1.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))
model1.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))
model1.add(Dense(len(char_list)+1,activation='softmax'))

model1.compile(optimizer=Adam(learning_rate=0.0001),loss='category_crossentropy',accuracy=['metrics'])

In [9]:
inputs = Input(shape=(32,128,1))
 

conv1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
pool1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv1)
conv2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool1)
pool2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv2)
conv3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool2)
conv4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv3)
pool4 = MaxPool2D(pool_size=(2, 1))(conv4)
conv5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool4)
batchnorm5 = BatchNormalization()(conv5)
conv6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batchnorm5)
batchnorm6 = BatchNormalization()(conv6)
pool6 = MaxPool2D(pool_size=(2, 1))(batchnorm6)
conv7 = Conv2D(512, (2,2), activation = 'relu')(pool6)
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv7)
blstm1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm1)
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm2)
test_model = Model(inputs, outputs)

In [10]:
labels = Input(name='the_labels', shape=[max_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
 
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
 
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])


model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')
filepath="C:/bestocrmodel.h5"
checkpoint=ModelCheckpoint(filepath=filepath
                           ,monitor='accuracy',verbose=0,save_best_only=True,save_weights_only=False)
callbacks_list=[checkpoint]

In [14]:
train_img=np.array(train_img)
train_input_length=np.array(train_input_length)
train_label_length=np.array(train_label_length)

valid_img=np.array(valid_img)
valid_input_length=np.array(valid_input_length)
valid_label_length=np.array(valid_label_length)


In [16]:
from keras.models import load_model

In [18]:

batch_size = 256
epochs = 10
model.fit(x=[train_img,train_pad,train_input_length,train_label_length], y=np.zeros(len(train_img)), batch_size=batch_size, epochs = epochs, verbose = 1, callbacks = callbacks_list)


Epoch 1/10
160000/160000 [==============================] - 279s 2ms/step - loss: 27.1001
Epoch 2/10


C:\Users\joyje\anaconda3\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


160000/160000 [==============================] - 264s 2ms/step - loss: 16.9050
Epoch 3/10
160000/160000 [==============================] - 263s 2ms/step - loss: 5.1589
Epoch 4/10
160000/160000 [==============================] - 263s 2ms/step - loss: 3.7192
Epoch 5/10
160000/160000 [==============================] - 262s 2ms/step - loss: 3.0940
Epoch 6/10
160000/160000 [==============================] - 262s 2ms/step - loss: 2.7429
Epoch 7/10
160000/160000 [==============================] - 261s 2ms/step - loss: 2.4714
Epoch 8/10
160000/160000 [==============================] - 261s 2ms/step - loss: 2.3043
Epoch 9/10
160000/160000 [==============================] - 261s 2ms/step - loss: 2.1278
Epoch 10/10
160000/160000 [==============================] - 261s 2ms/step - loss: 1.9801


In [24]:
model.save("bestmodel")